In [3]:
from gliner import GLiNER

model = GLiNER.from_pretrained("EmergentMethods/gliner_large_news-v2.1")

/Users/jingmengli/Documents/projects/Adverse-News-Screening/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 87154.37it/s]
/Users/jingmengli/Documents/projects/Adverse-News-Screening/.conda/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
text = """
[title]Elizabeth Holmes vs. Sam Bankman-Fried: A Tale of Two CEO Scandals [snippet]Explore the rise and fall of Elizabeth Holmes and Sam Bankman-Fried—two CEOs who built billion-dollar companies before facing fraud charges.
"""

labels = ["person", "location", "date", "event", "legal action", "organization"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Elizabeth Holmes => person
Sam Bankman-Fried => person
Elizabeth Holmes => person
Sam Bankman-Fried => person
fraud charges => legal action


In [5]:
import os
from openai import AzureOpenAI

endpoint = "https://jingm-m98bbo3t-eastus2.openai.azure.com/"
model_name = "gpt-4o"
deployment = "gpt-4o-2"

subscription_key = "532itr5hqsmMemyJtIsccwd02qHG0VivooiY9z25uXBIgWUlFqaVJQQJ99BDACHYHv6XJ3w3AAAAACOGr6DQ"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """
            You are an expert in extracting named entities from financial news. I will be providing a set of entities and some news articles containing [title] and [snippet], and you will need to extract the entities from the articles. Multiple news articles will be seperated by a newline character.
            ## Constraints
            1. References to the same entity should be grouped together while all variations should be listed.
            2. For each article, the output should be in a dictionary. The dictionary should contain the entities identified from that article, and the fields should include each entity type, entity_name and variations.
            3. Return the exact same number of dictionaries as the number of articles nested in a list. The order of the results should follow the order of the articles.
            4. The output should be a JSON object. Only return the JSON object, nothing else. Do not include any other strings whatsoever, even it's for formatting.

            ## Sample Output
            {
                "COMPANY": [
                    {
                    "entity_name": "...",
                    "variations": [
                        "...",
                    ]
                    },
                    ...
                ],
                "PERSON": [
                    {
                    "entity_name": "...",
                    "variations": [
                        "...",
                    ]
                    },
                    ...
                ],
                "FINANCIAL_INSTITUTION": [...],
                "REGULATORY_BODY": [...],
                "PROTENTIAL_CRIME": [...],
                "LEGAL_ACTION": [...],
                "ENFORCEMENT_ACTION": [...],
                "LOCATION": [...],
                "SANCTION_ENTITY": [...],
                "SECTOR": [...],
                "REGULATION": [...]
                }
            
            ## Entities
            COMPANY: Public or private companies, including subsidiaries.
            PERSON: Individuals, especially executives, board members, or other key figures.
            FINANCIAL_INSTITUTION: Banks, investment firms, hedge funds, etc.
            REGULATORY_BODY: Agencies like SEC, MAS, FCA, etc.
            PROTENTIAL_CRIME: Terms indicating potential crime.
            LEGAL_ACTION: Mentions of lawsuits, indictments, probes, settlements.
            ENFORCEMENT_ACTION: Actions like fines, bans, license suspensions.
            LOCATION: Cities, countries—important for sanctions or jurisdiction context.
            SANCTION_ENTITY: Country or individual sanctioned or under restrictions.
            SECTOR: Industry or economic sector (e.g., "energy", "fintech").
            REGULATION: Names or identifiers of laws or policies (e.g., GDPR, FCPA, Dodd-Frank).
            """
        },
        {
            "role": "user",
            "content": """
            [title]Elizabeth Holmes vs. Sam Bankman-Fried: A Tale of Two CEO Scandals [snippet]Explore the rise and fall of Elizabeth Holmes and Sam Bankman-Fried—two CEOs who built billion-dollar companies before facing fraud charges.
            """,
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

[
    {
        "COMPANY": [
            {
                "entity_name": "Theranos",
                "variations": []
            },
            {
                "entity_name": "FTX",
                "variations": []
            }
        ],
        "PERSON": [
            {
                "entity_name": "Elizabeth Holmes",
                "variations": []
            },
            {
                "entity_name": "Sam Bankman-Fried",
                "variations": ["SBF"]
            }
        ],
        "FINANCIAL_INSTITUTION": [],
        "REGULATORY_BODY": [],
        "PROTENTIAL_CRIME": [
            {
                "entity_name": "fraud",
                "variations": ["fraud charges"]
            }
        ],
        "LEGAL_ACTION": [
            {
                "entity_name": "fraud charges",
                "variations": []
            }
        ],
        "ENFORCEMENT_ACTION": [],
        "LOCATION": [],
        "SANCTION_ENTITY": [],
        "SECTOR": [],
        "REG

In [9]:
type(response.choices[0].message)

openai.types.chat.chat_completion_message.ChatCompletionMessage

In [2]:
import pandas as pd

# load the sample data
data = {'Group': ['Short', 'Short', 'Moderate', 'Moderate', 'Tall'], 'Size': ['Small', 'Small', 'Medium', 'Small', 'Large']}
df = pd.DataFrame(data)
dfg = df.groupby(by=["Group", "Size"]).size().reset_index(name="Time")
dfg

,Group,Size,Time
0,Moderate,Medium,1
1,Moderate,Small,1
2,Short,Small,2
3,Tall,Large,1


In [ ]:
[
    {
        "COMPANY": [
            {
                "entity_name": "Theranos",
                "variations": []
            },
            {
                "entity_name": "FTX",
                "variations": []
            }
        ],
        "PERSON": [
            {
                "entity_name": "Elizabeth Holmes",
                "variations": []
            },
            {
                "entity_name": "Sam Bankman-Fried",
                "variations": ["SBF"]
            }
        ],
        "FINANCIAL_INSTITUTION": [],
        "REGULATORY_BODY": [],
        "PROTENTIAL_CRIME": [
            {
                "entity_name": "fraud",
                "variations": ["fraud charges"]
            }
        ],
        "LEGAL_ACTION": [
            {
                "entity_name": "fraud charges",
                "variations": []
            }
        ],
        "ENFORCEMENT_ACTION": [],
        "LOCATION": [],
        "SANCTION_ENTITY": [],
        "SECTOR": [],
        "REGULATION": []
    }
]